In [1]:
import yaml
from pystac import STAC_IO, read_file, Item, Catalog, CatalogType
from urllib.parse import urlparse
import requests
import os

## Stage-in remote STAC items as a local STAC Catalog

In [2]:
def my_read_method(uri):
    
    parsed = urlparse(uri)
    
    if parsed.scheme.startswith('http'):
        return requests.get(uri).text
    else:
        return STAC_IO.default_read_text_method(uri)

In [3]:
input_references = ['https://terradue-rtd.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720.json']

In [4]:
STAC_IO.read_text_method = my_read_method
    
catalogs = []

for index, input_reference in enumerate(input_references):

    items = []
    
    thing = read_file(input_reference)

    if isinstance(thing, Item):

        items.append(thing)

    elif isinstance(thing, Catalog):

        for item in thing.get_items():

            items.append(item)

    # create catalog
    catalog = Catalog(id=items[0].id,
              description='staged STAC catalog with {}'.format(items[0].id))

    catalog.add_items(items)

    catalog.normalize_and_save(root_href=items[0].id,
                               catalog_type=CatalogType.RELATIVE_PUBLISHED)

    catalog.describe()
    
    catalogs.append(os.path.dirname(catalog.get_self_href()))

* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


In [5]:
catalogs

['/home/jovyan/demo/cwl-workflows/raster/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720']

## Expert mode - multitemporal RGB composite

In [6]:
with open(r'scombi-do-raster.yml') as file:
    
    cwl_params = yaml.load(file, Loader=yaml.FullLoader)

In [7]:
cwl_params

{'input_reference': {'class': 'Directory',
  'path': '/Users/fbrito/Documents/scombi-do/docker_tmpxcy75c4k/'},
 'red-band': 'red',
 'green-band': 'green',
 'blue-band': 'blue',
 'aoi': 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))',
 'color': 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45'}

In [8]:
cwl_params['input_reference']['path'] = catalogs[0]

In [9]:
cwl_params

{'input_reference': {'class': 'Directory',
  'path': '/home/jovyan/demo/cwl-workflows/raster/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720'},
 'red-band': 'red',
 'green-band': 'green',
 'blue-band': 'blue',
 'aoi': 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))',
 'color': 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45'}

In [10]:
with open(r'scombi-do-raster.yml', 'w') as file:
    params = yaml.dump(cwl_params, file)

Open a Terminal and run the CWL document with:

```console
cd /home/jovyan/demo/cwl-workflows/raster
cwltool --no-container scombi-do-raster.cwl#scombi-do-raster scombi-do-raster.yml 
```

Use the `demo/cwl-result-viewer.ipynb` Jupyter Notebook to view the results